In [12]:
using OrdinaryDiffEq, CUDA, LinearAlgebra
u0 = rand(1000)
A = randn(1000, 1000)
f(du, u, p, t) = mul!(du, A, u)
f1(du, u, p, t) = mul!(du, cu(A), u)
prob = ODEProblem(f, u0, (0.0, 1.0))
prob1 = ODEProblem(f1, cu(u0), (0.0f0, 1.0f0))
@time sol = solve(prob, Tsit5());

  0.951009 seconds (1.21 M allocations: 68.954 MiB, 94.08% compilation time)


In [14]:
@time sol = solve(prob1,Tsit5());

  0.704380 seconds (28.08 k allocations: 1.086 GiB, 9.34% gc time)


In [6]:
using DiffEqGPU, OrdinaryDiffEq
function lorenz(du,u,p,t)
    du[1] = p[1]*(u[2]-u[1])
    du[2] = u[1]*(p[2]-u[3]) - u[2]
    du[3] = u[1]*u[2] - p[3]*u[3]
end

u0 = Float32[1.0;0.0;0.0]
tspan = (0.0f0,100.0f0)
p = [10.0f0,28.0f0,8/3f0]
prob = ODEProblem(lorenz,u0,tspan,p)
prob_func = (prob,i,repeat) -> remake(prob,p=rand(Float32,3).*p)
monteprob = EnsembleProblem(prob, prob_func = prob_func, safetycopy=false)
@time sol = solve(monteprob,Tsit5(),EnsembleGPUArray(),trajectories=10_000,saveat=1.0f0)

 37.537149 seconds (91.96 M allocations: 5.942 GiB, 4.26% gc time, 78.19% compilation time)


EnsembleSolution Solution of length 10000 with uType:
ODESolution{Float32, 2, uType, Nothing, Nothing, Vector{Float32}, rateType, ODEProblem{Vector{Float32}, Tuple{Float32, Float32}, true, Vector{Float32}, ODEFunction{true, typeof(lorenz), UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}, Tsit5{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, IType, DiffEqBase.DEStats} where {uType, rateType, IType}